In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install lightgbm==2.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.1.0
    Uninstalling lightgbm-4.1.0:
      Successfully uninstalled lightgbm-4.1.0


In [ ]:
!pip install xgboost

In [ ]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt


import random

import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [ ]:
DATA_PATH = '/content/drive/MyDrive/lg_aimers/data/'

In [ ]:
with open(DATA_PATH + 'train.csv', encoding='utf-8') as train_file:
    raw_train_df = pd.read_csv(train_file)

with open(DATA_PATH + 'test.csv', encoding='utf-8') as test_file:
    raw_test_df = pd.read_csv(test_file)

In [ ]:
train_x = raw_train_df.filter(regex='X') # Input : X Featrue
train_y = raw_train_df.filter(regex='Y') # Output : Y Feature

In [ ]:
test_x = raw_test_df.filter(regex='X')

# EDA

In [ ]:
x_feature_info = pd.read_csv("/content/drive/MyDrive/lg_aimers/data/meta/x_feature_info.csv")
x_feature_info.head(len(x_feature_info))

,Feature,설명
0,X_01,PCB 체결 시 단계별 누름량(Step 1)
1,X_02,PCB 체결 시 단계별 누름량(Step 2)
2,X_03,방열 재료 1 무게
3,X_04,1차 검사 통과 여부
4,X_05,PCB 체결 시 단계별 누름량(Step 3)
5,X_06,PCB 체결 시 단계별 누름량(Step 4)
6,X_07,방열 재료 1 면적
7,X_08,방열 재료 2 면적
8,X_09,방열 재료 3 면적
9,X_10,방열 재료 2 무게


In [ ]:
y_feature_info = pd.read_csv("/content/drive/MyDrive/lg_aimers/data/meta/y_feature_info.csv")
y_feature_info.head(len(y_feature_info))

,Feature,설명
0,Y_01,안테나 Gain 평균 (각도1)
1,Y_02,안테나 1 Gain 편차
2,Y_03,안테나 2 Gain 편차
3,Y_04,평균 신호대 잡음비
4,Y_05,안테나 Gain 평균 (각도2)
5,Y_06,신호대 잡음비 (각도1)
6,Y_07,안테나 Gain 평균 (각도3)
7,Y_08,신호대 잡음비 (각도2)
8,Y_09,신호대 잡음비 (각도3)
9,Y_10,신호대 잡음비 (각도4)


In [ ]:
y_feature_spec_info = pd.read_csv("/content/drive/MyDrive/lg_aimers/data/meta/y_feature_spec_info.csv")
y_feature_spec_info.head(len(y_feature_spec_info))

,Feature,최소,최대
0,Y_01,0.2,2.0
1,Y_02,0.2,2.1
2,Y_03,0.2,2.1
3,Y_04,7.0,19.0
4,Y_05,22.0,36.5
5,Y_06,-19.2,19.0
6,Y_07,2.4,4.0
7,Y_08,-29.2,-24.0
8,Y_09,-29.2,-24.0
9,Y_10,-30.6,-20.0


In [ ]:
train_x.describe(include = 'all')

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
count,39607.000000,39607.000000,39607.000000,39607.0,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,...,39607.0,39607.0,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000
mean,68.412040,103.320166,68.826354,1.0,102.337203,70.597211,29.407490,164.449320,225.397470,0.002449,...,1.0,1.0,16676.913639,130.774557,131.457418,138.587595,127.992420,128.014110,137.884711,128.441119
std,2.655983,0.000372,5.151167,0.0,0.548353,2.259820,7.338204,220.402444,66.734725,0.086255,...,0.0,0.0,8584.427420,5.989845,5.934313,6.469813,5.713153,5.433958,6.552101,5.444245
min,56.268000,103.320000,56.470000,1.0,101.774000,61.726000,14.140000,38.460000,37.580000,0.000000,...,1.0,1.0,3341.830000,21.800000,21.910000,23.100000,21.330000,21.340000,22.980000,21.410000
25%,66.465000,103.320000,65.070000,1.0,101.949000,68.864000,27.890000,105.990000,188.540000,0.000000,...,1.0,1.0,13093.930000,126.948571,127.669758,134.475401,124.374525,124.697879,133.744103,125.134713
50%,68.504000,103.320000,67.270000,1.0,102.006000,69.884000,28.840000,115.040000,234.450000,0.000000,...,1.0,1.0,15273.330000,130.723378,131.352856,138.525575,128.020010,128.098728,137.900952,128.414501
75%,69.524000,103.320000,71.770000,1.0,103.144000,71.923000,29.870000,132.620000,263.960000,0.000000,...,1.0,1.0,17560.180000,134.538998,135.170761,142.702448,131.618662,131.487423,142.073102,131.839764
max,84.820000,103.321000,89.170000,1.0,103.160000,87.219000,163.860000,2387.440000,637.490000,3.600000,...,1.0,1.0,114563.630000,162.619458,194.513195,173.438623,152.406630,175.052891,170.155980,155.277538


In [ ]:
train_y.describe(include = 'all')

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
count,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000
mean,1.353814,1.057267,1.014002,13.621191,31.290467,16.529382,3.155054,-26.294839,-26.308623,-22.400062,24.325061,-26.237762,-26.233869,-26.245868
std,0.356223,0.386266,0.361492,2.686632,2.543222,1.893014,0.418940,0.660537,0.653580,0.920952,0.830197,0.656329,0.655090,0.655989
min,0.017000,0.007000,0.017000,-0.331000,18.589000,-19.963000,0.502000,-29.652000,-29.523000,-31.119000,19.844000,-29.544000,-29.448000,-29.620000
25%,1.127500,0.793000,0.769000,11.822000,29.768000,16.146000,2.863000,-26.689000,-26.702000,-22.871000,23.836000,-26.630000,-26.624000,-26.640000
50%,1.349000,1.044000,0.998000,13.837000,31.710000,16.694000,3.126000,-26.254000,-26.266000,-22.275000,24.420000,-26.198000,-26.193000,-26.204000
75%,1.576000,1.300000,1.239000,15.626000,33.184000,17.164000,3.433500,-25.855000,-25.871000,-21.791000,24.911500,-25.799000,-25.794000,-25.809000
max,4.409000,3.998000,3.756000,98.794000,37.250000,18.998000,5.299000,-23.785000,-23.960000,-20.052000,26.703000,-23.722000,-23.899000,-23.856000


In [ ]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39607 entries, 0 to 39606
Data columns (total 56 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X_01    39607 non-null  float64
 1   X_02    39607 non-null  float64
 2   X_03    39607 non-null  float64
 3   X_04    39607 non-null  int64  
 4   X_05    39607 non-null  float64
 5   X_06    39607 non-null  float64
 6   X_07    39607 non-null  float64
 7   X_08    39607 non-null  float64
 8   X_09    39607 non-null  float64
 9   X_10    39607 non-null  float64
 10  X_11    39607 non-null  float64
 11  X_12    39607 non-null  float64
 12  X_13    39607 non-null  float64
 13  X_14    39607 non-null  float64
 14  X_15    39607 non-null  float64
 15  X_16    39607 non-null  float64
 16  X_17    39607 non-null  float64
 17  X_18    39607 non-null  float64
 18  X_19    39607 non-null  float64
 19  X_20    39607 non-null  float64
 20  X_21    39607 non-null  float64
 21  X_22    39607 non-null  float64
 22

In [ ]:
train_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39607 entries, 0 to 39606
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Y_01    39607 non-null  float64
 1   Y_02    39607 non-null  float64
 2   Y_03    39607 non-null  float64
 3   Y_04    39607 non-null  float64
 4   Y_05    39607 non-null  float64
 5   Y_06    39607 non-null  float64
 6   Y_07    39607 non-null  float64
 7   Y_08    39607 non-null  float64
 8   Y_09    39607 non-null  float64
 9   Y_10    39607 non-null  float64
 10  Y_11    39607 non-null  float64
 11  Y_12    39607 non-null  float64
 12  Y_13    39607 non-null  float64
 13  Y_14    39607 non-null  float64
dtypes: float64(14)
memory usage: 4.2 MB


# 파생변수
* 누름량 합산 : 7 ~ 9 합산
* 01,02,05,06 : 총 방열재료 면적 ( 도표먼적이 도포량에 따라 발열 성능에 영향)
* 평평함을 유지하는 것(?)이 중요하다고 생각 : 스크류 삽입 깊이 평균값 추가
* 레이돔과 안테나 간 개별거리는 반드시 균일해야함
* 레이돔 치수 평균 값

In [ ]:
#누름량합산
train_x['sum7_9'] = train_x['X_07'] + train_x['X_08'] + train_x['X_09']
test_x['sum7_9'] = test_x['X_07'] + test_x['X_08'] + test_x['X_09']

In [ ]:
#총 방열재료 면적
train_x['sum_1_2_5_6'] = train_x['X_01'] + train_x['X_02'] + train_x['X_05'] + train_x['X_06']
test_x['sum_1_2_5_6'] = test_x['X_01'] + test_x['X_02'] + test_x['X_05'] + test_x['X_06']

In [ ]:
#스크류 삽입깊이 평균
train_x['mean_19_20_21_22'] = train_x[['X_19', 'X_20', 'X_21', 'X_22']].mean(axis=1)
test_x['mean_19_20_21_22'] = test_x[['X_19', 'X_20', 'X_21', 'X_22']].mean(axis=1)

In [ ]:
#레이돔 치수 평균
train_x['mean_41_42_43_44'] = train_x[['X_41', 'X_42', 'X_43', 'X_44']].mean(axis=1)
test_x['mean_41_42_43_44'] = test_x[['X_41', 'X_42', 'X_43', 'X_44']].mean(axis=1)

# 전처리
* 필요없는 열 제거
* 통과여부 열(4,23,47,48)
* 0값이 너무 많아 제거 (10,11)
* 정상/비정상 라벨 부여


In [ ]:
drop_col = ["X_04", "X_23", "X_47", "X_48"]
train_x = train_x.drop(drop_col,axis=1)
test_x = test_x.drop(drop_col,axis=1)

In [ ]:
drop_col2 = ["X_10", "X_11"]
train_x = train_x.drop(drop_col2,axis=1)
test_x = test_x.drop(drop_col2,axis=1)

In [ ]:
# 레이블 생성
labels = []
for i, row in train_y.iterrows():
    label = 0  # 일단 정상(0)으로 설정
    for feature, min_val, max_val in zip(y_feature_spec_info['Feature'], y_feature_spec_info['최소'], y_feature_spec_info['최대']):
        if not (min_val <= row[feature] <= max_val):
            label = 1  # 비정상(1)으로 변경
            break
    labels.append(label)

# 레이블 열 추가
train_y['label'] = labels

In [ ]:
train_y.head()

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14,label
0,2.056,1.456,1.680,10.502,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304,1
1,1.446,1.184,1.268,18.507,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438,0
2,1.251,0.665,0.782,14.082,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370,0
3,1.464,1.079,1.052,16.975,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345,0
4,0.983,0.646,0.689,15.047,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974,0


# AutoML
* Multioutput 사용 x
* 단일 항목(Y1 ~ Y14)까지 변수 중요도(Feature Importance) 확인 및 성능이 좋은 모델 확인

In [ ]:
!pip install pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 46.6

In [ ]:
from pycaret.regression import *

In [ ]:
#Concat Data
train_01 = pd.concat([train_x, train_y['Y_01']], axis=1)
train_02 = pd.concat([train_x, train_y['Y_02']], axis=1)
train_03 = pd.concat([train_x, train_y['Y_03']], axis=1)
train_04 = pd.concat([train_x, train_y['Y_04']], axis=1)
train_05 = pd.concat([train_x, train_y['Y_05']], axis=1)
train_06 = pd.concat([train_x, train_y['Y_06']], axis=1)
train_07 = pd.concat([train_x, train_y['Y_07']], axis=1)
train_08 = pd.concat([train_x, train_y['Y_08']], axis=1)
train_09 = pd.concat([train_x, train_y['Y_09']], axis=1)
train_10 = pd.concat([train_x, train_y['Y_10']], axis=1)
train_11 = pd.concat([train_x, train_y['Y_11']], axis=1)
train_12 = pd.concat([train_x, train_y['Y_12']], axis=1)
train_13 = pd.concat([train_x, train_y['Y_13']], axis=1)
train_14 = pd.concat([train_x, train_y['Y_14']], axis=1)

In [ ]:
model = setup(train_01, target = 'Y_01', session_id=42 )

,Description,Value
0,Session id,42
1,Target,Y_01
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


In [ ]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.2682,0.1204,0.3470,0.0462,0.1527,0.2821,18.2780
rf,Random Forest Regressor,0.2685,0.1205,0.3471,0.0457,0.1526,0.2778,78.2360
lightgbm,Light Gradient Boosting Machine,0.2687,0.1205,0.3471,0.0454,0.1526,0.2793,2.3150
catboost,CatBoost Regressor,0.2689,0.1209,0.3476,0.0428,0.1528,0.2802,14.6440
et,Extra Trees Regressor,0.2689,0.1209,0.3477,0.0424,0.1530,0.2826,28.9510
lr,Linear Regression,0.2706,0.1224,0.3498,0.0309,0.1539,0.2859,0.6380
ridge,Ridge Regression,0.2706,0.1224,0.3499,0.0301,0.1540,0.2863,0.0740
br,Bayesian Ridge,0.2708,0.1226,0.3501,0.0292,0.1541,0.2864,0.2300
ada,AdaBoost Regressor,0.2739,0.1247,0.3531,0.0124,0.1558,0.2887,7.2130
omp,Orthogonal Matching Pursuit,0.2741,0.1256,0.3543,0.0054,0.1559,0.2892,0.1150


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

GradientBoostingRegressor(random_state=42)

In [ ]:
model = setup(train_02, target = 'Y_02', session_id=42 )

,Description,Value
0,Session id,42
1,Target,Y_02
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


In [ ]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.2974,0.1426,0.3777,0.0387,0.1874,0.5404,18.1760
rf,Random Forest Regressor,0.2980,0.1427,0.3778,0.0381,0.1876,0.5372,79.3150
lightgbm,Light Gradient Boosting Machine,0.2977,0.1430,0.3781,0.0366,0.1875,0.5364,2.6860
catboost,CatBoost Regressor,0.2980,0.1430,0.3782,0.0361,0.1876,0.5339,14.6130
et,Extra Trees Regressor,0.2989,0.1436,0.3789,0.0324,0.1882,0.5439,28.6920
lr,Linear Regression,0.2989,0.1438,0.3793,0.0306,0.1883,0.5460,0.1750
ridge,Ridge Regression,0.2991,0.1439,0.3794,0.0300,0.1883,0.5476,0.1110
br,Bayesian Ridge,0.2993,0.1441,0.3796,0.0289,0.1885,0.5484,0.2080
omp,Orthogonal Matching Pursuit,0.3031,0.1475,0.3840,0.0062,0.1907,0.5508,0.1030
en,Elastic Net,0.3041,0.1483,0.3851,0.0005,0.1913,0.5504,0.0970


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

GradientBoostingRegressor(random_state=42)

In [ ]:
model = setup(train_03, target = 'Y_03', session_id=42 )

,Description,Value
0,Session id,42
1,Target,Y_03
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


In [ ]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.2789,0.1258,0.3546,0.0351,0.1786,0.4047,17.9860
lightgbm,Light Gradient Boosting Machine,0.2789,0.1259,0.3548,0.0341,0.1786,0.4027,3.1250
rf,Random Forest Regressor,0.2792,0.1260,0.3549,0.0337,0.1788,0.4045,77.4670
et,Extra Trees Regressor,0.2796,0.1263,0.3554,0.0312,0.1791,0.4059,28.8600
catboost,CatBoost Regressor,0.2795,0.1266,0.3557,0.0291,0.1790,0.4022,14.7880
lr,Linear Regression,0.2802,0.1270,0.3564,0.0254,0.1796,0.4085,0.1940
ridge,Ridge Regression,0.2803,0.1271,0.3565,0.0249,0.1797,0.4090,0.0900
br,Bayesian Ridge,0.2805,0.1273,0.3567,0.0236,0.1798,0.4096,0.1440
omp,Orthogonal Matching Pursuit,0.2839,0.1300,0.3605,0.0029,0.1818,0.4150,0.0670
en,Elastic Net,0.2841,0.1302,0.3608,0.0014,0.1820,0.4158,0.0650


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

GradientBoostingRegressor(random_state=42)

In [ ]:
model = setup(train_04, target = 'Y_04', session_id=42 )

,Description,Value
0,Session id,42
1,Target,Y_04
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


In [ ]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,2.0666,6.4939,2.5480,0.0731,0.1900,0.1723,84.5590
catboost,CatBoost Regressor,2.0626,6.4987,2.5490,0.0724,0.1901,0.1721,15.7290
lightgbm,Light Gradient Boosting Machine,2.0751,6.5484,2.5587,0.0653,0.1909,0.1734,3.2030
et,Extra Trees Regressor,2.0809,6.5627,2.5614,0.0633,0.1909,0.1734,29.7390
gbr,Gradient Boosting Regressor,2.0922,6.6288,2.5744,0.0539,0.1921,0.1750,19.7440
ridge,Ridge Regression,2.1177,6.7707,2.6018,0.0336,0.1940,0.1771,0.1810
lr,Linear Regression,2.1170,6.7716,2.6020,0.0335,0.1940,0.1770,1.4280
br,Bayesian Ridge,2.1180,6.7716,2.6020,0.0335,0.1940,0.1771,0.2160
lar,Least Angle Regression,2.1220,6.8216,2.6116,0.0261,0.1946,0.1773,0.1690
xgboost,Extreme Gradient Boosting,2.1190,6.8942,2.6254,0.0160,0.1949,0.1762,1.3980


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

RandomForestRegressor(n_jobs=-1, random_state=42)

In [ ]:
model = setup(train_05, target = 'Y_05', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_05
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.9767,6.2025,2.4902,0.0376,0.0803,0.0655,15.6600
lightgbm,Light Gradient Boosting Machine,1.9846,6.2316,2.4961,0.0330,0.0805,0.0658,1.9850
gbr,Gradient Boosting Regressor,1.9896,6.2518,2.5001,0.0300,0.0807,0.0660,19.1990
rf,Random Forest Regressor,2.0022,6.2765,2.5050,0.0261,0.0807,0.0662,88.9280
et,Extra Trees Regressor,2.0042,6.2864,2.5070,0.0246,0.0808,0.0663,30.5450
lr,Linear Regression,2.0006,6.3048,2.5107,0.0218,0.0810,0.0664,0.1320
br,Bayesian Ridge,2.0017,6.3087,2.5114,0.0212,0.0810,0.0664,0.2560
ridge,Ridge Regression,2.0023,6.3118,2.5120,0.0207,0.0810,0.0664,0.1130
omp,Orthogonal Matching Pursuit,2.0217,6.4279,2.5350,0.0027,0.0818,0.0671,0.1170
en,Elastic Net,2.0236,6.4406,2.5375,0.0007,0.0818,0.0671,0.1260


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = setup(train_06, target = 'Y_06', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_06
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.7413,3.2141,1.7857,0.0961,0.0717,0.0448,14.4220
lightgbm,Light Gradient Boosting Machine,0.7576,3.2431,1.7939,0.0861,0.0761,0.0458,3.2510
rf,Random Forest Regressor,0.7489,3.3864,1.8334,0.0471,0.0627,0.0452,98.1050
et,Extra Trees Regressor,0.7427,3.4254,1.8419,0.0419,0.0527,0.0448,29.5410
gbr,Gradient Boosting Regressor,0.7409,3.5037,1.8640,0.0176,0.0582,0.0447,19.2210
lr,Linear Regression,0.7252,3.5507,1.8749,0.0089,0.0469,0.0437,0.1710
ridge,Ridge Regression,0.7232,3.5519,1.8751,0.0088,0.0467,0.0436,0.1040
br,Bayesian Ridge,0.7212,3.5596,1.8770,0.0069,0.0465,0.0435,0.2130
omp,Orthogonal Matching Pursuit,0.7182,3.5800,1.8822,0.0016,0.0461,0.0433,0.1040
en,Elastic Net,0.7142,3.5856,1.8836,0.0003,0.0459,0.0430,0.0980


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = setup(train_07, target = 'Y_07', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_07
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.3249,0.1671,0.4088,0.0425,0.0999,0.1065,15.1840
rf,Random Forest Regressor,0.3257,0.1677,0.4095,0.0395,0.1001,0.1069,82.4680
lightgbm,Light Gradient Boosting Machine,0.3259,0.1679,0.4097,0.0384,0.1001,0.1069,2.3940
et,Extra Trees Regressor,0.3265,0.1687,0.4108,0.0334,0.1004,0.1072,30.4180
gbr,Gradient Boosting Regressor,0.3268,0.1689,0.4110,0.0323,0.1005,0.1072,19.5100
lr,Linear Regression,0.3279,0.1701,0.4124,0.0255,0.1008,0.1076,0.1810
ridge,Ridge Regression,0.3279,0.1702,0.4125,0.0253,0.1008,0.1076,0.1090
br,Bayesian Ridge,0.3280,0.1702,0.4126,0.0249,0.1009,0.1077,0.1530
lar,Least Angle Regression,0.3288,0.1712,0.4137,0.0194,0.1011,0.1079,0.0790
omp,Orthogonal Matching Pursuit,0.3327,0.1744,0.4176,0.0010,0.1021,0.1093,0.0730


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = setup(train_08, target = 'Y_08', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_08
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.4861,0.3930,0.6269,0.0965,0.0229,0.0184,14.9530
rf,Random Forest Regressor,0.4873,0.3933,0.6271,0.0956,0.0229,0.0185,81.8230
lightgbm,Light Gradient Boosting Machine,0.4872,0.3949,0.6284,0.0920,0.0229,0.0185,2.5300
et,Extra Trees Regressor,0.4897,0.3971,0.6301,0.0871,0.0230,0.0186,29.7510
gbr,Gradient Boosting Regressor,0.4893,0.3982,0.6310,0.0846,0.0230,0.0186,18.9020
ridge,Ridge Regression,0.4979,0.4109,0.6410,0.0554,0.0234,0.0189,0.1070
lr,Linear Regression,0.4979,0.4109,0.6410,0.0553,0.0234,0.0189,0.1210
br,Bayesian Ridge,0.4980,0.4111,0.6411,0.0551,0.0234,0.0189,0.2130
xgboost,Extreme Gradient Boosting,0.5013,0.4163,0.6452,0.0428,0.0235,0.0190,1.1790
lar,Least Angle Regression,0.5036,0.4198,0.6478,0.0351,0.0236,0.0191,0.1240


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = setup(train_09, target = 'Y_09', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_09
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.4839,0.3869,0.6220,0.0891,0.0227,0.0183,15.4530
lightgbm,Light Gradient Boosting Machine,0.4838,0.3872,0.6222,0.0884,0.0227,0.0183,2.6250
rf,Random Forest Regressor,0.4846,0.3873,0.6223,0.0881,0.0227,0.0184,80.3170
et,Extra Trees Regressor,0.4864,0.3901,0.6246,0.0815,0.0228,0.0184,30.5460
gbr,Gradient Boosting Regressor,0.4859,0.3909,0.6252,0.0796,0.0228,0.0184,18.7820
ridge,Ridge Regression,0.4943,0.4035,0.6352,0.0502,0.0231,0.0187,0.1070
lr,Linear Regression,0.4943,0.4035,0.6352,0.0501,0.0232,0.0187,0.1850
br,Bayesian Ridge,0.4943,0.4036,0.6353,0.0498,0.0232,0.0187,0.1470
lar,Least Angle Regression,0.4993,0.4112,0.6412,0.0320,0.0234,0.0189,0.1130
xgboost,Extreme Gradient Boosting,0.5003,0.4123,0.6421,0.0292,0.0234,0.0190,1.3410


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = setup(train_10, target = 'Y_10', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_10
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.6289,0.7363,0.8579,0.1331,0.0356,0.0278,15.0900
lightgbm,Light Gradient Boosting Machine,0.6316,0.7417,0.8611,0.1265,0.0357,0.0279,2.3250
rf,Random Forest Regressor,0.6349,0.7486,0.8651,0.1186,0.0359,0.0281,81.3060
gbr,Gradient Boosting Regressor,0.6365,0.7620,0.8728,0.1030,0.0362,0.0281,18.6350
et,Extra Trees Regressor,0.6394,0.7627,0.8732,0.1021,0.0362,0.0282,27.9400
xgboost,Extreme Gradient Boosting,0.6546,0.7882,0.8876,0.0721,0.0369,0.0289,1.3560
lr,Linear Regression,0.6523,0.7965,0.8923,0.0625,0.0370,0.0288,0.1380
br,Bayesian Ridge,0.6522,0.7968,0.8924,0.0622,0.0370,0.0288,0.2210
ridge,Ridge Regression,0.6525,0.7977,0.8930,0.0611,0.0370,0.0288,0.1150
omp,Orthogonal Matching Pursuit,0.6690,0.8311,0.9115,0.0218,0.0378,0.0295,0.1200


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = setup(train_11, target = 'Y_11', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_11
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.6333,0.6555,0.8096,0.0454,0.0324,0.0263,15.1820
lightgbm,Light Gradient Boosting Machine,0.6353,0.6586,0.8114,0.0411,0.0325,0.0264,2.5820
rf,Random Forest Regressor,0.6377,0.6595,0.8120,0.0397,0.0325,0.0265,85.2320
et,Extra Trees Regressor,0.6399,0.6637,0.8146,0.0335,0.0326,0.0266,29.6420
gbr,Gradient Boosting Regressor,0.6387,0.6639,0.8147,0.0333,0.0326,0.0265,18.5770
lr,Linear Regression,0.6459,0.6766,0.8225,0.0149,0.0329,0.0268,0.1270
ridge,Ridge Regression,0.6461,0.6769,0.8226,0.0144,0.0329,0.0268,0.0740
br,Bayesian Ridge,0.6466,0.6778,0.8232,0.0131,0.0329,0.0269,0.2130
lar,Least Angle Regression,0.6482,0.6812,0.8252,0.0083,0.0330,0.0269,0.0780
omp,Orthogonal Matching Pursuit,0.6510,0.6869,0.8287,-0.0001,0.0331,0.0270,0.0760


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = setup(train_12, target = 'Y_12', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_12
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.4847,0.3887,0.6234,0.0917,0.0228,0.0184,14.3430
lightgbm,Light Gradient Boosting Machine,0.4848,0.3896,0.6242,0.0896,0.0228,0.0184,4.1760
rf,Random Forest Regressor,0.4854,0.3900,0.6245,0.0887,0.0228,0.0185,80.1690
et,Extra Trees Regressor,0.4875,0.3919,0.6260,0.0842,0.0229,0.0185,29.4080
gbr,Gradient Boosting Regressor,0.4874,0.3937,0.6274,0.0801,0.0229,0.0185,19.1800
lr,Linear Regression,0.4954,0.4061,0.6372,0.0513,0.0233,0.0188,0.1850
ridge,Ridge Regression,0.4954,0.4061,0.6372,0.0513,0.0233,0.0188,0.1110
br,Bayesian Ridge,0.4955,0.4062,0.6373,0.0510,0.0233,0.0188,0.2180
lar,Least Angle Regression,0.4962,0.4073,0.6381,0.0485,0.0233,0.0189,0.1140
xgboost,Extreme Gradient Boosting,0.4999,0.4131,0.6427,0.0345,0.0235,0.0190,1.1240


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = setup(train_13, target = 'Y_13', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_13
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.4838,0.3877,0.6226,0.0924,0.0227,0.0184,14.2410
rf,Random Forest Regressor,0.4851,0.3882,0.6231,0.0912,0.0228,0.0184,79.2550
lightgbm,Light Gradient Boosting Machine,0.4845,0.3887,0.6234,0.0902,0.0228,0.0184,3.2500
et,Extra Trees Regressor,0.4859,0.3906,0.6250,0.0856,0.0228,0.0185,28.5920
gbr,Gradient Boosting Regressor,0.4867,0.3924,0.6264,0.0816,0.0229,0.0185,17.7780
lr,Linear Regression,0.4947,0.4048,0.6362,0.0524,0.0232,0.0188,0.1840
ridge,Ridge Regression,0.4948,0.4048,0.6362,0.0524,0.0232,0.0188,0.1100
br,Bayesian Ridge,0.4949,0.4050,0.6363,0.0521,0.0233,0.0188,0.1500
xgboost,Extreme Gradient Boosting,0.4983,0.4100,0.6403,0.0401,0.0234,0.0189,1.2160
lar,Least Angle Regression,0.5025,0.4176,0.6460,0.0228,0.0236,0.0191,0.1180


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
model = setup(train_14, target = 'Y_14', session_id=42 )
compare_models()

,Description,Value
0,Session id,42
1,Target,Y_14
2,Target type,Regression
3,Original data shape,"(39607, 57)"
4,Transformed data shape,"(39607, 57)"
5,Transformed train set shape,"(27724, 57)"
6,Transformed test set shape,"(11883, 57)"
7,Numeric features,56
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.4852,0.3899,0.6243,0.0911,0.0228,0.0184,14.6960
rf,Random Forest Regressor,0.4865,0.3904,0.6248,0.0898,0.0228,0.0185,79.3700
lightgbm,Light Gradient Boosting Machine,0.4854,0.3907,0.6250,0.0891,0.0228,0.0184,3.8160
et,Extra Trees Regressor,0.4877,0.3928,0.6267,0.0843,0.0229,0.0185,28.3200
gbr,Gradient Boosting Regressor,0.4880,0.3945,0.6281,0.0803,0.0229,0.0185,18.5640
lr,Linear Regression,0.4963,0.4072,0.6380,0.0509,0.0233,0.0189,0.1240
ridge,Ridge Regression,0.4964,0.4072,0.6380,0.0509,0.0233,0.0189,0.1100
br,Bayesian Ridge,0.4966,0.4073,0.6381,0.0506,0.0233,0.0189,0.2150
xgboost,Extreme Gradient Boosting,0.5001,0.4130,0.6426,0.0371,0.0235,0.0190,1.2190
lar,Least Angle Regression,0.5003,0.4135,0.6430,0.0359,0.0235,0.0190,0.1170


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

# 모델링

In [ ]:
import numpy as np
import pandas as pd
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor  # XGBoost 추가
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
import os
import warnings
import time

warnings.filterwarnings(action='ignore', category=UserWarning)

In [ ]:
# Custom Loss Function - Normalized Root Mean Square Error (NRMSE)
def custom_loss(y_true, y_pred):
    residuals = y_true - y_pred
    nrmse = np.sqrt(np.mean(residuals**2)) / (np.max(y_true) - np.min(y_true))
    return nrmse

# 데이터 분할을 위한 Stratified K-Fold 설정
skf = StratifiedKFold(n_splits=6, random_state=42, shuffle=True)
n = 0

fold_target_pred = []
fold_score = []

# 파일 디렉토리 생성
model_dir = './model'
os.makedirs(model_dir, exist_ok=True)

In [ ]:
for train_index, valid_index in skf.split(train_x, train_y['label']):
    n += 1

    val_pred_name = []
    val_pred = []
    target_pred = []

    # 학습 데이터와 검증 데이터 준비
    X_train = train_x.iloc[train_index]
    y_train = train_y.iloc[train_index]
    X_valid = train_x.iloc[valid_index]
    y_valid = train_y.iloc[valid_index]

    # 테스트 데이터 복사
    X_test = test_x.copy()

    # 모델링
    # CatBoost 모델 (GPU 학습)
    start_time_cat = time.time()
    model_cat = CatBoostRegressor(
        verbose=200, learning_rate=0.02, random_seed=42, iterations=5000,
        loss_function='MultiRMSE', od_wait=200, depth=9,
        task_type="GPU", devices='0:1', boosting_type='Plain'  # GPU 학습 설정
    )

    model_cat.fit(X_train, y_train)

    end_time_cat = time.time()

    # 모델 저장
    cat_path = f'./model/cat_{n}'
    model_cat.save_model(cat_path)

    val_pred_name.append("CatBoostRegressor")
    val_pred.append(model_cat.predict(X_valid))
    target_pred.append(model_cat.predict(X_test))

    # LGBM 모델
    start_time_lgb = time.time()
    model_lgbm = MultiOutputRegressor(LGBMRegressor(
        n_estimators=2000, learning_rate=0.01, max_depth=16, min_child_samples=56,
        subsample=0.4, num_leaves=80, random_state=42, n_jobs=8
    ))

    model_lgbm.fit(X_train, y_train)
    end_time_lgb = time.time()

    val_pred_name.append("LGBMRegressor")
    val_pred.append(model_lgbm.predict(X_valid))
    target_pred.append(model_lgbm.predict(X_test))

    # 모델 저장 (직렬화)
    import joblib
    joblib.dump(model_lgbm, f'./model/lgbm_{n}.joblib')

    # XGBoost 모델 (GPU 학습)
    start_time_xgb = time.time()
    model_xgb = MultiOutputRegressor(XGBRegressor(
        n_estimators=2000, learning_rate=0.01, max_depth=9, random_state=42, n_jobs=8, tree_method='gpu_hist'
    ))

    model_xgb.fit(X_train, y_train)
    end_time_xgb = time.time()

    val_pred_name.append("XGBoostRegressor")
    val_pred.append(model_xgb.predict(X_valid))
    target_pred.append(model_xgb.predict(X_test))

    # 모델 저장 (직렬화)
    joblib.dump(model_xgb, f'./model/xgb_{n}.joblib')

    # 검증 예측 결과 평균화
    preds = np.array(val_pred[0])
    for i in range(1, len(val_pred)):
        preds += val_pred[i]
    preds = preds / len(val_pred)

    # 테스트 예측 결과 평균화
    target_preds = target_pred[0]
    for i in range(1, len(target_pred)):
        target_preds += target_pred[i]
    target_preds = target_preds / len(target_pred)

    fold_target_pred.append(target_preds)

    print(f"========== Fold {n} ==========")
    for i in range(len(val_pred)):
        y_valid_arr = y_valid.values
        val_pred_arr = val_pred[i]
        print(f"{val_pred_name[i]} model NRMSE: {custom_loss(y_valid_arr, val_pred_arr):.4f}")

    print(f'CAT 코드 실행 시간: {end_time_cat - start_time_cat:.1f} seconds')
    print(f'LGB 코드 실행 시간: {end_time_lgb - start_time_lgb:.1f} seconds')
    print(f'XGB 코드 실행 시간: {end_time_xgb - start_time_xgb:.1f} seconds')

# 이후에 결과를 사용할 수 있도록 저장
np.save('fold_target_pred.npy', fold_target_pred)

0:	learn: 4.6437982	total: 148ms	remaining: 12m 19s
200:	learn: 4.3432460	total: 12.5s	remaining: 4m 58s
400:	learn: 4.2003859	total: 20.5s	remaining: 3m 55s
600:	learn: 4.0860995	total: 25.7s	remaining: 3m 8s
800:	learn: 3.9802557	total: 33.4s	remaining: 2m 55s
1000:	learn: 3.8833279	total: 38.7s	remaining: 2m 34s
1200:	learn: 3.7897934	total: 46.5s	remaining: 2m 27s
1400:	learn: 3.6957742	total: 51.9s	remaining: 2m 13s
1600:	learn: 3.6082923	total: 59.5s	remaining: 2m 6s
1800:	learn: 3.5199958	total: 1m 5s	remaining: 1m 55s
2000:	learn: 3.4327148	total: 1m 11s	remaining: 1m 47s
2200:	learn: 3.3481662	total: 1m 19s	remaining: 1m 40s
2400:	learn: 3.2644884	total: 1m 26s	remaining: 1m 33s
2600:	learn: 3.1838082	total: 1m 32s	remaining: 1m 25s
2800:	learn: 3.0984298	total: 1m 39s	remaining: 1m 17s
3000:	learn: 3.0184369	total: 1m 46s	remaining: 1m 10s
3200:	learn: 2.9404092	total: 1m 52s	remaining: 1m 3s
3400:	learn: 2.8606373	total: 2m	remaining: 56.5s
3600:	learn: 2.7869647	total: 2m 5

In [ ]:
final_pred = np.array(fold_target_pred[0])

for i in range(1, 6):
    final_pred += fold_target_pred[i]

final_pred = final_pred/6

In [ ]:
final_pred

array([[  1.48307112,   1.33661087,   1.21177325, ..., -25.94684197,
        -25.99872293,   0.19522983],
       [  1.48324271,   1.26882204,   1.16704458, ..., -26.16647634,
        -26.18541034,   0.12555233],
       [  1.51717635,   1.17043563,   1.13271213, ..., -25.92575555,
        -25.92481512,   0.24690989],
       ...,
       [  1.22783659,   0.93006307,   0.95039092, ..., -26.44657878,
        -26.47440201,   0.09732576],
       [  1.18852672,   0.82550395,   0.90620078, ..., -26.35410487,
        -26.40495717,   0.08359646],
       [  1.32040115,   0.95743435,   1.03100617, ..., -26.59964072,
        -26.59475136,   0.09739963]])

In [ ]:
filename = '/content/drive/MyDrive/lg_aimers/data/sample_submission.csv'
sample_submission = pd.read_csv(filename)
sample_submission.head(5)

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,TEST_00002,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,TEST_00003,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,TEST_00004,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,TEST_00005,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
for idx, col in enumerate(sample_submission.columns):
    if col=='ID':
        continue
    sample_submission[col] = final_pred[:,idx-1]
sample_submission.head()

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.483071,1.336611,1.211773,14.071429,31.499809,16.656513,3.104292,-26.005208,-26.071499,-22.090559,24.689981,-25.962562,-25.946842,-25.998723
1,TEST_00002,1.483243,1.268822,1.167045,12.974974,30.772444,16.463041,3.182303,-26.221727,-26.231895,-22.469392,24.208519,-26.174798,-26.166476,-26.185410
2,TEST_00003,1.517176,1.170436,1.132712,14.783200,31.776732,16.870003,3.138663,-25.980262,-25.995241,-22.101883,24.561173,-25.915946,-25.925756,-25.924815
3,TEST_00004,1.436401,1.172905,1.044194,15.072770,32.657605,17.184763,3.063827,-25.603839,-25.656617,-21.690001,25.026169,-25.609519,-25.598606,-25.582238
4,TEST_00005,1.361267,1.020345,0.972228,15.191028,31.981814,17.081195,3.078677,-25.546934,-25.582432,-21.962637,24.822113,-25.471630,-25.501231,-25.471647


In [ ]:
sample_submission.to_csv('./final_pred.csv', index=False)

In [ ]:
import glob
model_paths = sorted(glob.glob('./model/*'))

import zipfile
submission = zipfile.ZipFile("./model.zip", 'w')
for path in model_paths:
    submission.write(path)
submission.close()